In [1]:
CUDA_LAUNCH_BLOCKING=1

In [2]:
import pandas as pd
import numpy as np 
import torch 
import io
import os
import math
import copy
import pickle
import zipfile
from textwrap import wrap
from pathlib import Path
from itertools import zip_longest
from collections import defaultdict
from urllib.error import URLError
from urllib.request import urlopen

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F 
from torch.optim.lr_scheduler import _LRScheduler
from typing import List, Dict
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
def setupSeed(seed: int = 1):
    lst : List = [np.random.seed, torch.manual_seed, torch.cuda.manual_seed]
    for l in lst :
        l(seed)

In [4]:
df = pd.read_csv("reviews_filtered.csv", usecols=["AuthorId", "RecipeId", "Rating"])
df = df[["AuthorId", "RecipeId", "Rating"]]
df.head()

,AuthorId,RecipeId,Rating
0,1634,4384,4
1,2046,4523,2
2,1773,7435,5
3,2085,44,5
4,2046,5221,4


In [5]:
df[df["AuthorId"] == 1634]

,AuthorId,RecipeId,Rating
0,1634,4384,4
863,1634,8909,3
896,1634,8729,2
900,1634,9115,5
1070,1634,5174,0
1287,1634,10783,0
1673,1634,11530,4
1691,1634,11641,4
1703,1634,11674,5
1705,1634,9982,4


In [6]:
def create_dataset(df_rating):
    unique_users: pd.Series = df_rating["AuthorId"].unique()
    author_to_indx: Dict = {
        authorId: indx for indx, authorId in enumerate(unique_users)
    }
    new_authors = df_rating["AuthorId"].map(author_to_indx)
    
    unique_recipes: pd.Series = df_rating["RecipeId"].unique()
    recipes_to_indx: Dict = {
        recipeId: indx for indx, recipeId in enumerate(unique_recipes)
    }
    new_recipes = df_rating["RecipeId"].map(recipes_to_indx)
    n_authors: int = unique_users.shape[0]
    n_recipes: int = unique_recipes.shape[0]

    x = pd.DataFrame({"AuthorId": new_authors, "RecipeId": new_recipes})
    y = df_rating["Rating"].astype(np.float32)
    #x = df[["AuthorId", "RecipeId"]]
    #y = df_rating["Rating"].astype(np.float32)

    return (x,y), (n_authors, n_recipes), (author_to_indx, recipes_to_indx)
(X, Y), (n, m), _ = create_dataset(df)

In [7]:
X[X["AuthorId"] == 2]

,AuthorId,RecipeId
2,2,2
9,2,9
87,2,84
251,2,207
260,2,216


In [8]:
class ReviewsIterator:
    
    def __init__(self, X, y, batch_size=32, shuffle=True):
        X, y = np.asarray(X), np.asarray(y)
        
        if shuffle:
            index = np.random.permutation(X.shape[0])
            X, y = X[index], y[index]
            
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_batches = int(math.ceil(X.shape[0] // batch_size))
        self._current = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()
    
    def next(self):
        if self._current >= self.n_batches:
            raise StopIteration()
        k = self._current
        self._current += 1
        bs = self.batch_size
        return self.X[k*bs:(k + 1)*bs], self.y[k*bs:(k + 1)*bs]
    
class ReviewsIterator:
    
    def __init__(self, X, y, batch_size=32, shuffle=True):
        X, y = np.asarray(X), np.asarray(y)
        
        if shuffle:
            index = np.random.permutation(X.shape[0])
            X, y = X[index], y[index]
            
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.n_batches = int(math.ceil(X.shape[0] // batch_size))
        self._current = 0
        
    def __iter__(self):
        return self
    
    def __next__(self):
        return self.next()
    
    def next(self):
        if self._current >= self.n_batches:
            raise StopIteration()
        k = self._current
        self._current += 1
        bs = self.batch_size
        return self.X[k*bs:(k + 1)*bs], self.y[k*bs:(k + 1)*bs]

In [9]:
def batches(X, y, bs=32, shuffle=True):
    for xb, yb in ReviewsIterator(X, y, bs, shuffle):
        xb = torch.LongTensor(xb)
        yb = torch.FloatTensor(yb)
        yield xb, yb.view(-1, 1) 
    

In [10]:
class EmbeddingNet(nn.Module):
    """
    Creates a dense network with embedding layers.
    
    Args:
    
        n_users:            
            Number of unique users in the dataset.

        n_movies: 
            Number of unique movies in the dataset.

        n_factors: 
            Number of columns in the embeddings matrix.

        embedding_dropout: 
            Dropout rate to apply right after embeddings layer.

        hidden:
            A single integer or a list of integers defining the number of 
            units in hidden layer(s).

        dropouts: 
            A single integer or a list of integers defining the dropout 
            layers rates applyied right after each of hidden layers.
            
    """
    def __init__(self, n_users, n_movies,
                 n_factors=50, embedding_dropout=0.02, 
                 hidden=10, dropouts=0.2):
        
        super().__init__()
        hidden = get_list(hidden)
        dropouts = get_list(dropouts)
        n_last = hidden[-1]
        
        def gen_layers(n_in):
            """
            A generator that yields a sequence of hidden layers and 
            their activations/dropouts.
            
            Note that the function captures `hidden` and `dropouts` 
            values from the outer scope.
            """
            nonlocal hidden, dropouts
            assert len(dropouts) <= len(hidden)
            
            for n_out, rate in zip_longest(hidden, dropouts):
                yield nn.Linear(n_in, n_out)
                yield nn.ReLU()
                if rate is not None and rate > 0.:
                    yield nn.Dropout(rate)
                n_in = n_out
            
        self.u = nn.Embedding(n_users, n_factors)
        self.m = nn.Embedding(n_movies, n_factors)
        self.drop = nn.Dropout(embedding_dropout)
        self.hidden = nn.Sequential(*list(gen_layers(n_factors * 2)))
        self.fc = nn.Linear(n_last, 1)
        self._init()
        
    def forward(self, users, movies, minmax=None):
        features = torch.cat([self.u(users), self.m(movies)], dim=1)
        x = self.drop(features)
        x = self.hidden(x)
        out = torch.sigmoid(self.fc(x))
        if minmax is not None:
            min_rating, max_rating = minmax
            out = out*(max_rating - min_rating + 1) + min_rating - 0.5
        return out
    
    def _init(self):
        """
        Setup embeddings and hidden layers with reasonable initial values.
        """
        
        def init(m):
            if type(m) == nn.Linear:
                torch.nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)
                
        self.u.weight.data.uniform_(-0.05, 0.05)
        self.m.weight.data.uniform_(-0.05, 0.05)
        self.hidden.apply(init)
        init(self.fc)
    
    
def get_list(n):
    if isinstance(n, (int, float)):
        return [n]
    elif hasattr(n, '__iter__'):
        return list(n)
    raise TypeError('layers configuraiton should be a single number or a list of numbers')

In [11]:
EmbeddingNet(n, m, n_factors=150, hidden=100, dropouts=0.5)

EmbeddingNet(
  (u): Embedding(1821, 150)
  (m): Embedding(2517, 150)
  (drop): Dropout(p=0.02, inplace=False)
  (hidden): Sequential(
    (0): Linear(in_features=300, out_features=100, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

In [12]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [13]:
def triangular(step_size, max_lr, method='triangular', gamma=0.99):
    
    def scheduler(epoch, base_lr):
        period = 2 * step_size
        cycle = math.floor(1 + epoch/period)
        x = abs(epoch/step_size - 2*cycle + 1)
        delta = (max_lr - base_lr)*max(0, (1 - x))

        if method == 'triangular':
            pass  # we've already done
        elif method == 'triangular2':
            delta /= float(2 ** (cycle - 1))
        elif method == 'exp_range':
            delta *= (gamma**epoch)
        else:
            raise ValueError('unexpected method: %s' % method)
            
        return base_lr + delta
        
    return scheduler

In [14]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + math.cos(math.pi*t/t_max))/2
    
    return scheduler

In [15]:
print(X)

      AuthorId  RecipeId
0            0         0
1            1         1
2            2         2
3            3         3
4            1         4
...        ...       ...
4005      1367       696
4006      1165       696
4007      1450       696
4008      1588       696
4009      1791       696

[4010 rows x 2 columns]


In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=0)
datasets = {'train': (X_train, y_train), 'val': (X_valid, y_valid)}
dataset_sizes = {'train': len(X_train), 'val': len(X_valid)}

In [17]:
X_valid

,AuthorId,RecipeId
1098,545,793
2408,306,1566
2436,1144,1585
3283,1583,2092
1366,910,538
...,...,...
2549,841,1655
1708,841,839
316,255,205
2052,1193,280


In [18]:
minmax = df["Rating"].min().astype(float), df["Rating"].max().astype(float)
minmax

(0.0, 5.0)

In [19]:
net = EmbeddingNet(
    n_users=n, n_movies=m, 
    n_factors=200, hidden=[500, 500, 500], 
    embedding_dropout=0.05, dropouts=[0.5, 0.5, 0.25])

In [20]:
lr = 1e-4
wd = 0.03
bs = 2000
n_epochs = 1000
patience = 250
no_improvements = 0
best_loss = np.inf
best_weights = None
history = []
lr_history = []

device = "cpu"#torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

net.to(device)
criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay=wd)
iterations_per_epoch = int(math.ceil(dataset_sizes['train'] // bs))
scheduler = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/10))

for epoch in range(n_epochs):
    stats = {'epoch': epoch + 1, 'total': n_epochs}
    
    for phase in ('train', 'val'):
        training = phase == 'train'
        running_loss = 0.0
        n_batches = 0
        
        for batch in batches(*datasets[phase], shuffle=training, bs=bs):
            x_batch, y_batch = [b.to(device) for b in batch]
            optimizer.zero_grad()
        
            # compute gradients only during 'train' phase
            with torch.set_grad_enabled(training):
                outputs = net(x_batch[:, 0], x_batch[:, 1], minmax)
                loss = criterion(outputs, y_batch)
                print(outputs, y_batch)
                # don't update weights and rates when in 'val' phase
                if training:
                    scheduler.step()
                    loss.backward()
                    optimizer.step()
                    lr_history.extend(scheduler.get_lr())
                    
            running_loss += loss.item()
            
        epoch_loss = running_loss / dataset_sizes[phase]
        stats[phase] = epoch_loss
        
        # early stopping: save weights of the best model so far
        if phase == 'val':
            if epoch_loss < best_loss:
                print('loss improvement on epoch: %d' % (epoch + 1))
                best_loss = epoch_loss
                best_weights = copy.deepcopy(net.state_dict())
                no_improvements = 0
            else:
                no_improvements += 1
                
    history.append(stats)
    print('[{epoch:03d}/{total:03d}] train: {train:.4f} - val: {val:.4f}'.format(**stats))
    if no_improvements >= patience:
        print('early stopping after epoch {epoch:03d}'.format(**stats))
        break

/mnt/Dev_partition/anaconda3/envs/universal/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([[2.6528],
        [2.6149],
        [2.6272],
        ...,
        [2.5584],
        [2.6385],
        [2.6291]], grad_fn=<SubBackward0>) tensor([[5.],
        [4.],
        [5.],
        ...,
        [5.],
        [5.],
        [0.]])
loss improvement on epoch: 1
[001/1000] train: 2.8084 - val: 0.0000
tensor([[2.6680],
        [2.6996],
        [2.6697],
        ...,
        [2.5796],
        [2.6217],
        [2.5209]], grad_fn=<SubBackward0>) tensor([[5.],
        [5.],
        [5.],
        ...,
        [4.],
        [0.],
        [5.]])
[002/1000] train: 2.6975 - val: 0.0000
tensor([[2.7875],
        [2.6271],
        [2.5626],
        ...,
        [2.6554],
        [2.6836],
        [2.6248]], grad_fn=<SubBackward0>) tensor([[5.],
        [4.],
        [5.],
        ...,
        [5.],
        [5.],
        [5.]])
[003/1000] train: 2.6351 - val: 0.0000
tensor([[2.6471],
        [2.7299],
        [2.5892],
        ...,
        [2.6601],
        [2.6525],
        [2.8132]], 

/mnt/Dev_partition/anaconda3/envs/universal/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


tensor([[2.6711],
        [2.7258],
        [2.8144],
        ...,
        [2.6443],
        [2.8422],
        [2.7600]], grad_fn=<SubBackward0>) tensor([[5.],
        [5.],
        [5.],
        ...,
        [5.],
        [4.],
        [0.]])
[005/1000] train: 2.5392 - val: 0.0000
tensor([[2.7391],
        [2.7671],
        [2.6939],
        ...,
        [2.7407],
        [2.5468],
        [2.8092]], grad_fn=<SubBackward0>) tensor([[5.],
        [4.],
        [5.],
        ...,
        [5.],
        [5.],
        [5.]])
[006/1000] train: 2.5038 - val: 0.0000
tensor([[2.6654],
        [2.7727],
        [2.8720],
        ...,
        [2.8184],
        [2.7191],
        [2.6087]], grad_fn=<SubBackward0>) tensor([[0.],
        [5.],
        [3.],
        ...,
        [3.],
        [4.],
        [5.]])
[007/1000] train: 2.4657 - val: 0.0000
tensor([[2.8074],
        [2.8709],
        [2.7870],
        ...,
        [2.7950],
        [2.8553],
        [2.8027]], grad_fn=<SubBackward0>) tenso

In [ ]:
datasets["val"]

(      AuthorId  RecipeId
 1098       545       793
 2408       306      1566
 2436      1144      1585
 3283      1583      2092
 1366       910       538
 ...        ...       ...
 2549       841      1655
 1708       841       839
 316        255       205
 2052      1193       280
 3714      1740      2166
 
 [802 rows x 2 columns],
 1098    5.0
 2408    4.0
 2436    5.0
 3283    5.0
 1366    4.0
        ... 
 2549    5.0
 1708    5.0
 316     4.0
 2052    2.0
 3714    1.0
 Name: Rating, Length: 802, dtype: float32)

In [ ]:
groud_truth, predictions = [], []
def custom_round(x):
    decimal_part = x - np.floor(x)
    if decimal_part > 0.75:
        return np.ceil(x)
    else:
        return np.round(x)

with torch.no_grad():
    for batch in batches(*datasets['val'], shuffle=False, bs=1):
        print(batch)
        x_batch, y_batch = [b.to(device) for b in batch]
        outputs = net(x_batch[:, 0], x_batch[:, 1], minmax)
        groud_truth.extend(y_batch.tolist())
        predictions.extend(outputs.tolist())

groud_truth = np.asarray(groud_truth).ravel()
predictions = np.round(np.asarray(predictions).ravel())
final_loss = np.sqrt(np.mean(predictions - groud_truth)**2)
print(f'Final RMSE: {final_loss:.4f}')

(tensor([[545, 793]]), tensor([[5.]]))
(tensor([[ 306, 1566]]), tensor([[4.]]))
(tensor([[1144, 1585]]), tensor([[5.]]))
(tensor([[1583, 2092]]), tensor([[5.]]))
(tensor([[910, 538]]), tensor([[4.]]))
(tensor([[1186,  714]]), tensor([[4.]]))
(tensor([[ 490, 1751]]), tensor([[5.]]))
(tensor([[ 560, 1727]]), tensor([[5.]]))
(tensor([[1200, 1434]]), tensor([[4.]]))
(tensor([[1283, 1536]]), tensor([[4.]]))
(tensor([[ 450, 2327]]), tensor([[5.]]))
(tensor([[960, 282]]), tensor([[4.]]))
(tensor([[398, 375]]), tensor([[4.]]))
(tensor([[ 992, 1029]]), tensor([[5.]]))
(tensor([[1449, 1874]]), tensor([[5.]]))
(tensor([[1764,  792]]), tensor([[5.]]))
(tensor([[ 508, 2170]]), tensor([[5.]]))
(tensor([[147, 160]]), tensor([[4.]]))
(tensor([[ 976, 1661]]), tensor([[5.]]))
(tensor([[ 978, 1018]]), tensor([[5.]]))
(tensor([[667, 682]]), tensor([[0.]]))
(tensor([[1704, 2292]]), tensor([[5.]]))
(tensor([[788, 835]]), tensor([[5.]]))
(tensor([[1418, 2041]]), tensor([[4.]]))
(tensor([[515,  63]]), tensor(

In [ ]:
predictions

array([5., 4., 4., 5., 5., 5., 2., 5., 1., 5., 4., 2., 3., 5., 2., 4., 4.,
       5., 3., 2., 1., 3., 3., 5., 5., 2., 5., 4., 2., 5., 5., 2., 5., 2.,
       3., 5., 5., 5., 5., 4., 5., 4., 5., 4., 4., 3., 1., 4., 4., 3., 5.,
       5., 4., 4., 2., 4., 2., 5., 3., 3., 4., 4., 3., 3., 1., 3., 4., 5.,
       4., 4., 4., 5., 1., 5., 5., 5., 5., 4., 5., 5., 4., 3., 5., 5., 3.,
       5., 4., 4., 1., 1., 3., 5., 4., 5., 4., 5., 5., 5., 4., 5., 4., 4.,
       1., 2., 4., 4., 1., 4., 5., 3., 5., 5., 3., 4., 5., 1., 5., 5., 5.,
       1., 5., 5., 4., 5., 3., 4., 5., 5., 5., 4., 5., 2., 4., 5., 5., 3.,
       3., 5., 5., 4., 5., 5., 4., 4., 5., 3., 5., 3., 4., 4., 3., 1., 4.,
       5., 4., 3., 4., 3., 2., 2., 1., 4., 5., 5., 4., 3., 5., 5., 5., 3.,
       4., 2., 2., 5., 5., 3., 5., 4., 5., 5., 2., 2., 3., 4., 3., 4., 4.,
       5., 2., 3., 4., 4., 5., 5., 4., 4., 3., 5., 5., 4., 4., 5., 4., 2.,
       2., 4., 2., 4., 3., 4., 4., 5., 4., 1., 5., 3., 5., 3., 4., 2., 5.,
       5., 5., 5., 5., 4.

In [ ]:
groud_truth

array([5., 4., 5., 5., 4., 4., 5., 5., 4., 4., 5., 4., 4., 5., 5., 5., 5.,
       4., 5., 5., 0., 5., 5., 4., 5., 4., 5., 5., 5., 5., 4., 5., 5., 4.,
       4., 5., 5., 5., 5., 5., 5., 1., 5., 0., 5., 4., 0., 5., 5., 3., 5.,
       5., 4., 5., 5., 5., 5., 5., 4., 5., 5., 5., 4., 4., 1., 5., 5., 4.,
       3., 5., 5., 5., 4., 4., 5., 0., 0., 5., 0., 4., 5., 5., 5., 5., 5.,
       4., 5., 5., 5., 5., 4., 5., 0., 4., 5., 5., 5., 4., 5., 5., 5., 5.,
       0., 0., 0., 5., 5., 5., 4., 4., 4., 0., 5., 5., 5., 5., 5., 0., 5.,
       5., 5., 5., 5., 3., 4., 3., 4., 5., 5., 4., 5., 5., 5., 5., 5., 5.,
       4., 5., 0., 5., 5., 5., 5., 5., 5., 5., 5., 4., 0., 3., 5., 0., 2.,
       5., 5., 0., 4., 0., 4., 0., 0., 5., 5., 4., 3., 5., 2., 5., 5., 0.,
       5., 5., 5., 5., 3., 5., 5., 4., 5., 3., 5., 5., 5., 5., 5., 5., 5.,
       0., 3., 5., 5., 4., 5., 5., 5., 5., 3., 5., 0., 5., 4., 5., 5., 5.,
       3., 5., 5., 5., 4., 5., 3., 5., 4., 0., 0., 5., 1., 5., 5., 5., 5.,
       4., 5., 5., 5., 5.

In [ ]:
predictions

array([5., 4., 4., 5., 5., 5., 2., 5., 1., 5., 4., 2., 3., 5., 2., 4., 4.,
       5., 3., 2., 1., 3., 3., 5., 5., 2., 5., 4., 2., 5., 5., 2., 5., 2.,
       3., 5., 5., 5., 5., 4., 5., 4., 5., 4., 4., 3., 1., 4., 4., 3., 5.,
       5., 4., 4., 2., 4., 2., 5., 3., 3., 4., 4., 3., 3., 1., 3., 4., 5.,
       4., 4., 4., 5., 1., 5., 5., 5., 5., 4., 5., 5., 4., 3., 5., 5., 3.,
       5., 4., 4., 1., 1., 3., 5., 4., 5., 4., 5., 5., 5., 4., 5., 4., 4.,
       1., 2., 4., 4., 1., 4., 5., 3., 5., 5., 3., 4., 5., 1., 5., 5., 5.,
       1., 5., 5., 4., 5., 3., 4., 5., 5., 5., 4., 5., 2., 4., 5., 5., 3.,
       3., 5., 5., 4., 5., 5., 4., 4., 5., 3., 5., 3., 4., 4., 3., 1., 4.,
       5., 4., 3., 4., 3., 2., 2., 1., 4., 5., 5., 4., 3., 5., 5., 5., 3.,
       4., 2., 2., 5., 5., 3., 5., 4., 5., 5., 2., 2., 3., 4., 3., 4., 4.,
       5., 2., 3., 4., 4., 5., 5., 4., 4., 3., 5., 5., 4., 4., 5., 4., 2.,
       2., 4., 2., 4., 3., 4., 4., 5., 4., 1., 5., 3., 5., 3., 4., 2., 5.,
       5., 5., 5., 5., 4.

In [ ]:

groud_truth, predictions = [], []
with torch.no_grad():
    for batch in batches(*datasets['train'], shuffle=False, bs=1):
        print(batch)
        x_batch, y_batch = [b.to(device) for b in batch]
        outputs = net(x_batch[:, 0], x_batch[:, 1], minmax)
        groud_truth.extend(y_batch.tolist())
        predictions.extend(outputs.tolist())

groud_truth = np.asarray(groud_truth).ravel()
predictions = np.round(np.asarray(predictions).ravel())
final_loss = np.sqrt(np.mean(predictions - groud_truth)**2)
print(f'Final RMSE: {final_loss:.4f}')

(tensor([[841, 417]]), tensor([[5.]]))
(tensor([[ 123, 1073]]), tensor([[5.]]))
(tensor([[1589, 1369]]), tensor([[5.]]))
(tensor([[848, 897]]), tensor([[5.]]))
(tensor([[187,   7]]), tensor([[4.]]))
(tensor([[240,  61]]), tensor([[5.]]))
(tensor([[ 490, 2456]]), tensor([[3.]]))
(tensor([[ 577, 1394]]), tensor([[5.]]))
(tensor([[1119, 1145]]), tensor([[5.]]))
(tensor([[120, 139]]), tensor([[1.]]))
(tensor([[156, 166]]), tensor([[5.]]))
(tensor([[492, 469]]), tensor([[4.]]))
(tensor([[1048, 1117]]), tensor([[4.]]))
(tensor([[ 538, 2511]]), tensor([[4.]]))
(tensor([[1261,  119]]), tensor([[5.]]))
(tensor([[1, 5]]), tensor([[5.]]))
(tensor([[ 433, 2504]]), tensor([[5.]]))
(tensor([[1064,  615]]), tensor([[5.]]))
(tensor([[1406,  792]]), tensor([[5.]]))
(tensor([[841, 632]]), tensor([[2.]]))
(tensor([[680, 964]]), tensor([[4.]]))
(tensor([[ 791, 1041]]), tensor([[5.]]))
(tensor([[803, 852]]), tensor([[5.]]))
(tensor([[ 490, 2238]]), tensor([[4.]]))
(tensor([[1749, 2353]]), tensor([[5.]]))
(

In [ ]:
#torch.save(net.state_dict(), "DL_Rmse_0.2.pth")